http://www.litster.org/blog/2010/05/30/python-and-exif-metadata-theres-more-than-one-way-to-do-it/

http://piexif.readthedocs.io/en/latest/functions.html

In [1]:
import os
import datetime as dt
import re
import piexif
from PIL.ExifTags import TAGS

In [2]:
def list_dir_jpeg(basedir=os.path.curdir):
    jpeg_files = []
    other_files = []
    sub_dirs = []
    for item in os.listdir(basedir):
        abs_path = os.path.join(basedir,item)
        if os.path.isfile(abs_path):
            if re.search('(\.JPG)',item):
                jpeg_files.append(abs_path)
            else:
                other_files.append(abs_path)
        else:
            sub_dirs.append(abs_path)
    return (jpeg_files,other_files,sub_dirs)

In [3]:
def modify_jpeg_exif_datetimes(filename,td):
    exif_dict = piexif.load(filename)
    for key in exif_dict.keys():
        if type(exif_dict[key]) == dict:
            for item in exif_dict[key].keys():
                tag = TAGS.get(item,item)
                if 'Date' in str(tag):
                    old_date_string = exif_dict[key][item]
                    old_date_dt = dt.datetime.strptime(old_date_string,"%Y:%m:%d %H:%M:%S")
                    ndt = old_date_dt + dt.timedelta(days=td[0],hours=td[1],minutes=td[2],seconds=td[3])
                    new_date_string = '{:%Y:%m:%d %H:%M:%S}'.format(ndt)
                    exif_dict[key][item] = new_date_string
    piexif.insert(piexif.dump(exif_dict),filename)


In [4]:
def modify_jpeg_dates(directory=os.curdir,days=0,hours=0,minutes=0,seconds=0):
    td = [days,hours,minutes,seconds]
    jpegs,others,subdirs = list_dir_jpeg(directory)
    
    print '=JPEG-EXIF-DATETIME-MODIFIER'+ '='*12
    print 'Ignored files:', len(others)
    print 'Ignored sub directories:', len(subdirs)
    print 'JPEG files to be edited:', len(jpegs)
    print '='*40
    if len(jpegs) == 0:
        print 'There are no "*.JPG" files in supplied directory!'
        print '='*40
    else:
        print 'JPEG Metadata to be offset as follows:'
        print 'Days:'+str(td[0])+'|'+'Hours:'+str(td[1])+'|'+'Minutes:'+str(td[2])+'|'+'Seconds:'+str(td[3])
        print '='*40
        print 'Editing JPEG Metadata...'
        for pic in jpegs:
            print ' '*2 + os.path.basename(pic)
            modify_jpeg_exif_datetimes(pic,td)
        print '='*40
        print 'Editing Complete!', str(len(jpegs)), 'files modified.'
        print '='*40

In [5]:
modify_jpeg_dates(directory=os.path.curdir)

=JPEG-EXIF-DATETIME-MODIFIER============
Ignored files: 5
Ignored sub directories: 2
JPEG files to be edited: 0
There are no "*.JPG" files in supplied directory!


In [66]:
def modify_jpeg_filename(directory=os.curdir):
    jpegs,others,subdirs = list_dir_jpeg(directory)
    
    print '=JPEG-FILENAME-MODIFIER'+ '='*17
    print 'Ignored files:', len(others)
    print 'Ignored sub directories:', len(subdirs)
    print 'JPEG files to be edited:', len(jpegs)
    print '='*40
    if len(jpegs) == 0:
        print 'There are no "*.JPG" files in supplied directory!'
        print '='*40
    else:
        print 'Editing JPEG Filenames...'
        for pic in jpegs:
            fdt = dt.datetime.strptime(piexif.load(pic)['0th'][306],"%Y:%m:%d %H:%M:%S")
            new_filename = 'IMG_'+'{:%Y%m%d_%H%M%S}'.format(fdt)+'.JPG'
            try:
                os.rename(pic,directory+'\\'+new_filename)
            except WindowsError, arg: 
                if re.search('[a-zA-Z]{3}_[0-9]{8}_[0-9]{6}.JPG',new_filename):
                    new_filename = 'IMG_'+'{:%Y%m%d_%H%M%S}'.format(fdt)+'_1'+'.JPG'
                    os.rename(pic,directory+'\\'+new_filename)
                elif re.search('[a-zA-Z]{3}_[0-9]{8}_[0-9]{6}_([0-9]+?).JPG',new_filename):
                    filenum = int(re.search('[a-zA-Z]{3}_[0-9]{8}_[0-9]{6}_([0-9]+?).JPG',new_filename).match(1))
                    new_filename = 'IMG_'+'{:%Y%m%d_%H%M%S}'.format(fdt)+'_'+str(filenum + 1)+'.JPG'
                    #os.rename(pic,directory+'\\'+new_filename)
                else:
                    raise WindowsError, arg
            print ' '*2 + os.path.basename(pic) + ' --> ' + new_filename
            #modify_jpeg_exif_datetimes(pic,td)
        print '='*40
        print 'Editing Complete!', str(len(jpegs)), 'files modified.'
        print '='*40

In [69]:
modify_jpeg_filename('C:\Users\Skrzym\Documents\Data\Data Projects\PhotoMetadataEditing\me')

=JPEG-FILENAME-MODIFIER=================
Ignored files: 8
Ignored sub directories: 1
JPEG files to be edited: 22
Editing JPEG Filenames...
  IMG_20140317_204347_1.JPG --> IMG_20140317_204347_1.JPG
  IMG_20140317_204358.JPG --> IMG_20140317_204358.JPG
  IMG_20140317_204404.JPG --> IMG_20140317_204404.JPG
  IMG_20140719_152629.JPG --> IMG_20140719_152629.JPG
  IMG_20140731_154129.JPG --> IMG_20140731_154129.JPG
  IMG_20160604_162235.JPG --> IMG_20160604_162235.JPG
  IMG_20160604_181442.JPG --> IMG_20160604_181442.JPG
  IMG_20160604_181605.JPG --> IMG_20160604_181605.JPG
  IMG_20160604_181613.JPG --> IMG_20160604_181613.JPG
  IMG_20160604_181619.JPG --> IMG_20160604_181619.JPG
  IMG_20160604_181641.JPG --> IMG_20160604_181641.JPG
  IMG_20160604_181643.JPG --> IMG_20160604_181643.JPG
  IMG_20160604_181651.JPG --> IMG_20160604_181651.JPG
  IMG_20160604_181843.JPG --> IMG_20160604_181843.JPG
  IMG_20160604_181850.JPG --> IMG_20160604_181850.JPG
  IMG_20160604_181939.JPG --> IMG_20160604_1819

In [31]:
help(os.rename)

Help on built-in function rename in module nt:

rename(...)
    rename(old, new)
    
    Rename a file or directory.



In [132]:
def modify_jpeg_filename(directory=os.curdir):
    jpegs,others,subdirs = list_dir_jpeg(directory)
    for pic in jpegs:
            oldpic = pic[:-4]+'_old'+pic[-4:]
            os.rename(pic,oldpic)
    jpegs,others,subdirs = list_dir_jpeg(directory)
    
    print '=JPEG-FILENAME-MODIFIER'+ '='*17
    print 'Ignored files:', len(others)
    print 'Ignored sub directories:', len(subdirs)
    print 'JPEG files to be edited:', len(jpegs)
    print '='*40
    if len(jpegs) == 0:
        print 'There are no "*.JPG" files in supplied directory!'
        print '='*40
    else:
        print 'Editing JPEG Filenames...'
        
        for pic in jpegs:
            
            fdt = dt.datetime.strptime(piexif.load(pic)['0th'][306],"%Y:%m:%d %H:%M:%S")
            
            matches = [
                item 
                for item in os.listdir(directory) 
                if re.search('IMG_'+'{:%Y%m%d_%H%M%S}'.format(fdt)+'(_?[0-9]*)\.[jJ][pP][gG]',item)
            ]
            if len(matches) > 0:
                if os.path.basename(pic) != 'IMG_'+'{:%Y%m%d_%H%M%S}'.format(fdt)+'.JPG':
                    new_filename = 'IMG_'+'{:%Y%m%d_%H%M%S}'.format(fdt)+'_'+str(len(matches))+'.JPG'
            else:
                new_filename = 'IMG_'+'{:%Y%m%d_%H%M%S}'.format(fdt)+'.JPG'
                
            try:
                os.rename(pic,directory+'\\'+new_filename)
            except WindowsError, arg:
                print(os.path.basename(pic),new_filename)
                raise WindowsError, arg
            print ' '*2 + os.path.basename(pic)[:-8] + ' --> ' + new_filename[:-4]
            #modify_jpeg_exif_datetimes(pic,td)
        print '='*40
        print 'Editing Complete!', str(len(jpegs)), 'files modified.'
        print '='*40

In [135]:
modify_jpeg_filename('C:\Users\Skrzym\Documents\Data\Data Projects\PhotoMetadataEditing\me')

=JPEG-FILENAME-MODIFIER=================
Ignored files: 8
Ignored sub directories: 1
JPEG files to be edited: 1050
Editing JPEG Filenames...
  IMG_20140317_204347_1 - Copy (10) - Copy - Copy --> IMG_20140317_204347_1
  IMG_20140317_204347_1 - Copy (10) - Copy --> IMG_20140317_204347_2
  IMG_20140317_204347_1 - Copy (11) - Copy - Copy --> IMG_20140317_204347_3
  IMG_20140317_204347_1 - Copy (11) - Copy --> IMG_20140317_204347_4
  IMG_20140317_204347_1 - Copy (12) - Copy --> IMG_20140317_204347_5
  IMG_20140317_204347_1 - Copy (13) - Copy --> IMG_20140317_204347_6
  IMG_20140317_204347_1 - Copy (14) - Copy --> IMG_20140317_204347_7
  IMG_20140317_204347_1 - Copy (15) - Copy --> IMG_20140317_204347_8
  IMG_20140317_204347_1 - Copy (16) - Copy --> IMG_20140317_204347_9
  IMG_20140317_204347_1 - Copy (17) - Copy --> IMG_20140317_204347_10
  IMG_20140317_204347_1 - Copy (18) - Copy --> IMG_20140317_204347_11
  IMG_20140317_204347_1 - Copy (2) - Copy - Copy --> IMG_20140317_204347_12
  IMG_20